# Step 2: ANALYZE

In [17]:
import pandas as pd
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0
import re
import pandas as pd
import spacy
from spacy.lang.en.stop_words import STOP_WORDS as SP_en
from spacy.lang.es.stop_words import STOP_WORDS as SP_es

In [ ]:
#!python -m spacy download en
#nlp = spacy.load('en')

#!python -m spacy download es_core_news_md
#nlp_es = spacy.load('es')

## Getting csv

df_jobs.to_csv(f'..\\data\\raw\\df_{job_title_df}_{location}_{now_file}.csv',
              sep = ';',
              encoding= 'ANSI',
              index=False,
              )

In [14]:
dic_key_words = \
{   'data analyst'  : ['sql',
                       'python',
                       'business intelligence', 'bi' ,
                       'excel',
                       'dashboard', 'reporting',
                       'tableau', 'powerbi', 'power bi', 'qlik', 'sportfire',
                       'business analyst'
                      ],
 
    'data scientist': ['sql', 
                       'python', 'r', 
                       'ml', 'machine learning','dl','deep learning', 'nn','neuronal networks', 
                       'pandas', 'numpy', 
                       'data visualization', 'matplotlib', 'seaborn', 
                       'spss',  'advanced statistics','statistics',
                       'sklearn','sk learn', 'scikit learn', 'scikit-learn' ,'tensorflow', 'keras', 
                       'data wrangling', 'nlp', 'a/b testing','a b testing'
                      ],
 
    'data engineer':  ['sql', 'nosql', 'no sql', 
                       'python', 'r', 'scala', 'java', 'spark', 'hadoop', 'hive', 'impala', 'kafka', 
                       'data wrangling', 'etl','extract, transform and load', 'map reduce','mapreduce', 'data acquisition',
                       'big data', 'bigdata', 'data warehouse', 'aws',
                       'software engineering'
                      ]
}

In [2]:
df_profile = pd.read_csv(f'..\\data\\raw\\df_profile.csv',
                  sep = ';',
                  encoding= 'utf8'
              )
df_profile

,Unnamed: 0,info
0,profile,cristopherrojaslepe
1,headline,data scientist & analyst | python | pandas | n...
2,open new jobs,True
3,about,data scientist & analyst and industrial engine...
4,experiences,"['collection coordination analyst', 'real esta..."
5,years of experiences,"['1 yr 2 mos', '1 yr', '2 yrs 1 mo', '8 mos', ..."
6,total years,7
7,degrees,"[""master's degree"", 'bachelor of engineering -..."
8,fields,"['data analytics', 'industrial engineering']"
9,skills,"['analytical skills', 'pandas', 'team manageme..."


In [3]:
df_jobs = pd.read_csv(f'..\\data\\raw\\df_jobs_Madrid_2020-04-16-20.47.csv',
              sep = ';',
              encoding= 'utf8',
              )

In [7]:
df_jobs.head(3)

,JOB TITLE,LOCATION,SEARCH DATETIME,Current Job Id,Job html,Job name,Company name,Company location,Posted date,Estimated post date,Easy apply,Skills match,Seniority Level,Industry,Employment Type,Job Functions,Job info
0,data analyst,Madrid,2020-04-16-20:47,1744889236,https://www.linkedin.com/jobs/view/1744889236/,Analista de datos,YUKON IT SPAIN & EUROPE,Madrid y alrededores,1 month,2020-03-19-20:47,True,NaN,Associate,Information Technology & Services Telecommunic...,Full-time,Information Technology Engineering,Posted Date\nPosted 1 month ago\nNumber of vie...
1,data analyst,Madrid,2020-04-16-20:47,1822454033,https://www.linkedin.com/jobs/view/1822454033/,Backoffice Product Owner,SoYou_Bankia,"Madrid y alrededores, España",1 day,2020-04-15-20:47,True,NaN,Associate,Financial Services,Full-time,Analyst Information Technology Product Management,Posted Date\nPosted 1 day ago\nNumber of views...
2,data analyst,Madrid,2020-04-16-20:47,1766737805,https://www.linkedin.com/jobs/view/1766737805/,Analyst Intern- Group Growth Department,Interbrand,"Madrid, ES",1 week,2020-04-09-20:47,False,NaN,NaN,Graphic Design Management Consulting Marketing...,Full-time,Marketing Sales,Posted Date\nPosted 1 week ago\nNumber of view...


In [5]:
# ONLY IN CASE TO CORRECT WRONG DF
df_jobs.drop(columns='Job htmlJob name', inplace=True)

In [6]:
df_jobs = df_jobs[['JOB TITLE', 'LOCATION', 'SEARCH DATETIME', 'Current Job Id',
       'Job html', 'Job name', 'Company name', 'Company location',
       'Posted date', 'Estimated post date', 'Easy apply', 'Skills match',
       'Seniority Level', 'Industry', 'Employment Type', 'Job Functions',
       'Job info']]

In [8]:
df_jobs['Job name'].head(10)

0                                    Analista de datos
1                             Backoffice Product Owner
2              Analyst Intern- Group Growth Department
3                   Business Data & Operations Analyst
4                   Analista De Datos (Python), Madrid
5    Data Analyst -Scientist - Uber Eats (Sector es...
6                                       Data Analytics
7                                  Python Data Analyst
8                        Data Analytics Manager Madrid
9                                    Analista de Datos
Name: Job name, dtype: object

In [9]:
s = df_jobs.loc[3,'Job info']
s

'Posted Date\nPosted 5 months ago\nNumber of views\n422 views\nShare\nShow more options\nSave\nSave Business Data & Operations Analyst at Enghouse Interactive\nApply\nSee how you compare to 66 applicants. Reactivate Premium\nJob\n66 applicants\nAssociate\nCompany\n501-1,000 employees\nComputer Software\nConnections\nYou have 0 connections at this company.\nAdd >\nEstamos buscando un Business Data & Operations Analyst reportando al Managing Director, para asumir la gestión de procesos y datos en las áreas de Marketing, Ventas, Proyectos, Soporte y Producto.\n\nEl puesto requerirá que la persona sea capaz de hacer un análisis profundo de los datos y proponer mejoras en los procesos de las áreas mencionadas, para ayudar fijar KPIs y dar soporte a la toma de decisiones, con el objetivo de incrementar los ingresos y la productividad.\n\nAsí mismo, ayudará al Managing Director en la preparación de informes trimestrales para el BOD y revisiones trimestrales/mensuales del negocio con el Manage

In [10]:
detect(s)

'es'

In [11]:
df_jobs['Info Language'] = df_jobs['Job info'].apply(lambda x: detect(x))

In [12]:
df_jobs['Info Language'].unique()

array(['es', 'en'], dtype=object)

In [13]:
df_jobs['Seniority Level'].unique()

array(['Associate', nan, 'Entry level', 'Mid-Senior level', 'Director',
       'Internship', 'Executive'], dtype=object)

In [ ]:
def coincidencias(job):
    for s in dic_key_words.get(job):
        if 
        

In [ ]:
df_jobs['DA coincidences'] = df_jobs['Job info'].apply(lambda x: detect(x))